In [1]:
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# get URL
driver = webdriver.Chrome()
baseURL = "https://www.glassdoor.com/index.htm"
driver.get(baseURL)
driver.implicitly_wait(3)

In [3]:
# Search job and location
keywordElement = driver.find_element_by_id("KeywordSearch")
locationElement = driver.find_element_by_id("LocationSearch")
keywordElement.clear()
keywordElement.send_keys("Data Analyst")
locationElement.clear()
locationElement.send_keys("New York, NY")
driver.find_element_by_class_name("gd-btn-mkt").click()

In [4]:
# get text page by page 
# jobsList = []
# jobsDscript = []

# page_remaining = True
# while page_remaining:

try:       
    emailMeJob_pop = driver.find_element_by_class_name("mfp-close")
    emailMeJob_pop.click()
except:
    current_html = driver.page_source
    bs = BeautifulSoup(current_html,"html.parser")
    
    jobs_title =bs.select("td.job_title")
    companies_name = bs.select("td.company")
    jobs_location = bs.select("td.location")
    jobs_salary = bs.select("td.salary")
    companies_star = bs.select("span.compactStars ")
    #find 30 joblinks under tag<td class="job_title">, if use find_all(a class_="jobLink) there will be 90 hyperlin
    jobLinks = bs.select("td.job_title > a") 
    




#     next_page = driver.find_element_by_class_name("next")
#     try:
#         next_page.find_element_by_tag_name("a")
#         next_page.click()

#     except:
#         page_remaining = False
#         print("Finished")

In [5]:
# for indx, value in enumerate(jobsDscript):
#     if value != "NA":
#         print(indx, "ok")
#     else:
#         print(indx, value)

In [6]:

# pattern = re.compile(r"\u2022\s[A-z]*.*")
# bullets = re.finditer(pattern,var)
# for bullet in bullets:
#     print(bullet[0])



In [15]:
var_list = [jobs_title,companies_name,jobs_location,jobs_salary,companies_star,jobLinks]
print(list(map(len,var_list)))



[30, 30, 30, 30, 30, 30]


In [8]:
driver.close()